In [10]:
import os 
import pandas as pd

In [2]:
%pwd

'e:\\ML_Projects_iNeuron\\iNeuron_Project_Census_data_Classification_With_MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\ML_Projects_iNeuron\\iNeuron_Project_Census_data_Classification_With_MLflow'

In [84]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class Data_preprocessing_ValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    Preprocess_data: Path
    

In [85]:
from Mlflow_Ineuron_Project.constants import *
from Mlflow_Ineuron_Project.utils.common import read_yaml, create_directories

In [86]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> Data_preprocessing_ValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])
        create_directories([config.preprocess_data_path])

        data_validation_config = Data_preprocessing_ValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
            Preprocess_data=config.preprocess_data_path
            
        )

        return data_validation_config

In [87]:
import os
from Mlflow_Ineuron_Project import logger
from  sklearn.preprocessing import LabelEncoder 

In [88]:
class Data_preprocessing_Validation:
    def __init__(self, config: Data_preprocessing_ValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
       # remove the extra space around the columns 
            data.columns=data.columns.str.strip()

            numerical_col=['age','fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']

            categorical_col=['workclass', 'education', 'marital-status', 'occupation', 'sex', 'country','salary']
            # for col in categorical_col:
            #     if col in data.columns:
            #         data[col] = data[col].str.lower()
            #     if col in data.columns:
            #         data[col] = data[col].str.lower() 
            # print(data.head())
            data.drop(columns=['relationship', 'race','education-num'], inplace=True)

            # label encoding 
            for i in categorical_col:
                le=LabelEncoder()
                data[i]=le.fit_transform(data[i])
            
            
            
            # print(data.info())

            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            print(all_schema)
            for col in all_cols:
                print(col)
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")


            data.to_csv(os.path.join(self.config.Preprocess_data,"preprocessed_data.csv"),index=False)

            return validation_status
        
        except Exception as e:
            raise e



In [89]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation =Data_preprocessing_Validation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-04-17 12:35:28,569: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-17 12:35:28,572: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-17 12:35:28,575: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-17 12:35:28,575: INFO: common: created directory at: artifacts]
[2025-04-17 12:35:28,584: INFO: common: created directory at: artifacts/data_validation]
[2025-04-17 12:35:28,586: INFO: common: created directory at: artifacts/data_preprocessed]
dict_keys(['age', 'workclass', 'fnlwgt', 'education', 'marital-status', 'occupation', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'country', 'salary'])
age
workclass
fnlwgt
education
marital-status
occupation
sex
capital-gain
capital-loss
hours-per-week
country
salary
